# Rapid Eye Movements (REMs) detection

This notebook demonstrates how to use YASA to automatically detect rapid eye movements (REMs) on EOG data.

Please make sure to install the latest version of YASA first by typing the following line in your terminal or command prompt:

`pip install --upgrade yasa`

In [2]:
import sys
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import mne
from mne.filter import filter_data
sns.set(font_scale=1.2)
import yasa

## Data loading

Let's load 50 seconds of EOG data (LOC and ROC) during REM sleep, sampled at 256 Hz.

In [15]:
# Load data
subject_label = 'R12_2005'
channels_label = ['EOG LOC-A2', 'EOG ROC-A2']
raw = mne.io.read_raw_edf(f'C:/Users/klacourse/Documents/NGosselin/data/edf/RBD/10_first_subjects/{subject_label}.edf', preload=True)
sf=256.0
# Select a subset of EEG channels
raw.pick(channels_label)


Extracting EDF parameters from C:\Users\klacourse\Documents\NGosselin\data\edf\RBD\10_first_subjects\R12_2005.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 8098047  =      0.000 ... 31632.996 secs...


<RawEDF | R12_2005.edf, 2 x 8098048 (31633.0 s), ~123.6 MB, data loaded>

### Restraining the detection to REM sleep

In [16]:
import pandas as pd
hypno = pd.read_csv(f"C:/Users/klacourse/Documents/NGosselin/data/edf/RBD/10_first_subjects/{subject_label}_hypno.tsv", skiprows=0, header=None)
hypno = np.squeeze(hypno.values)

In [17]:
hypno_up = yasa.hypno_upsample_to_data(hypno, sf_hypno=1/30, data=raw._data[0,:], sf_data=sf)

09-Oct-24 10:03:14 | WARNING | Hypnogram is SHORTER than data by 13.00 seconds. Padding hypnogram with last value to match data.size.


In [18]:
from yasa import rem_detect

loc = raw._data[0,:] * 1e6
roc = raw._data[1,:] * 1e6
rem = rem_detect(loc, roc, sf, hypno=hypno_up, include=5, amplitude=(50, 325), duration=(0.3, 1.5), freq_rem=(0.5, 5), remove_outliers=False, verbose=False)

In [11]:
rem.summary().round(3)

,Start,Peak,End,Duration,LOCAbsValPeak,ROCAbsValPeak,LOCAbsRiseSlope,ROCAbsRiseSlope,LOCAbsFallSlope,ROCAbsFallSlope,Stage
0,24667.871,24668.062,24668.738,0.867,90.852,98.477,453.722,532.109,121.995,91.465,4
1,24696.562,24696.703,24697.184,0.621,51.051,87.426,470.069,320.712,132.447,138.960,4
2,24697.184,24697.410,24698.148,0.965,72.804,239.639,265.789,1148.910,99.977,322.016,4
3,24776.273,24776.652,24776.973,0.699,58.647,90.301,183.305,203.702,178.196,300.383,4
4,24802.105,24802.551,24803.301,1.195,54.647,45.922,119.715,105.007,70.662,63.503,4
...,...,...,...,...,...,...,...,...,...,...,...
85,26680.625,26680.832,26681.531,0.906,76.718,107.784,388.219,494.440,109.642,154.753,4
86,27616.656,27617.340,27617.645,0.988,100.653,84.968,162.453,112.842,327.374,283.709,4
87,27617.645,27618.180,27618.805,1.160,89.555,96.879,169.037,178.274,114.994,124.332,4
88,27629.703,27630.160,27630.770,1.066,53.500,55.059,107.511,127.574,77.527,99.164,4


In [12]:
rem.summary(grp_stage=True)

,Count,Density,Duration,LOCAbsValPeak,ROCAbsValPeak,LOCAbsRiseSlope,ROCAbsRiseSlope,LOCAbsFallSlope,ROCAbsFallSlope
Stage,,,,,,,,,
4,90,4.186047,0.944358,77.228483,90.837027,258.593113,299.176643,175.921458,245.873384


In [20]:
# Save the REMs dataFrame in a tsv file
rems_detection_df = rem.summary().round(3)
rems_detection_df.to_csv(f"C:/Users/klacourse/Documents/NGosselin/data/edf/RBD/10_first_subjects/{subject_label}_YASA_REMs_summary.tsv", sep='\t')
# Modify the DataFrame to be compatible with Snooz
#   Snooz dataframe : [group, name, start_sec, duration_sec, channels]
#   Define 
#       group as "YASA",
#       name as "YASA_REM", 
#       start_time as rems_detection_df['start'],
#       duration as rems_detection_df['duration'],
#       channel as channels_label
snooz_rem = pd.DataFrame({
    'group': 'YASA',
    'name': 'YASA_REM',
    'start_sec': rems_detection_df['Start'],
    'duration_sec': rems_detection_df['Duration'],
    'channels': [channels_label] * len(rems_detection_df)
})
# Save the REMs dataFrame in a tsv file
snooz_rem.to_csv(f"C:/Users/klacourse/Documents/NGosselin/data/edf/RBD/10_first_subjects/{subject_label}_YASA_REMs_snooz.tsv", sep='\t', index=False)

### Computation time

In [ ]:
%timeit rem_detect(loc, roc, sf)

In [ ]:
# Line profiling
# %load_ext line_profiler
# %lprun -f rem_detect rem_detect(loc, roc, sf)